## Building a Dall-e type model

---

In [ ]:
## pip installs 
! pip install datasets transformers torch torchvision 

#### 1. Data Preparation

---

In [1]:
## Download the dataset as a whole
from datasets import load_dataset

dataset_name = "HuggingFaceM4/COCO"

dataset = load_dataset(dataset_name, trust_remote_code=True)

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

COCO.py:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

The repository for HuggingFaceM4/COCO contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HuggingFaceM4/COCO.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [15]:
fraction= 0.1

small_train = dataset["train"].shuffle(seed=42).select(range(int(len(dataset["train"]) * fraction)))
small_test = dataset["test"].shuffle(seed=42).select(range(int(len(dataset["test"]) * fraction)))
small_validation = dataset["validation"].shuffle(seed=42).select(range(int(len(dataset["validation"]) * fraction)))

print(f"Small dataset train size: {len(small_train)}")
print(f"Small dataset test size: {len(small_test)}")
print(f"Small dataset validation size: {len(small_validation)}")

Small dataset train size: 56674
Small dataset test size: 2501
Small dataset validation size: 2501


In [16]:
##@ Saving them...
small_train.save_to_disk("small_train")
small_test.save_to_disk("small_test")
small_validation.save_to_disk("small_validation")

Saving the dataset (0/19 shards):   0%|          | 0/56674 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2501 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2501 [00:00<?, ? examples/s]

In [18]:
!zip small_train.zip /kaggle/working/small_train

  adding: kaggle/working/small_train/ (stored 0%)


In [19]:
!zip small_test.zip /kaggle/working/small_test

  adding: kaggle/working/small_test/ (stored 0%)


In [20]:
!zip small_validation.zip /kaggle/working/small_validation

  adding: kaggle/working/small_validation/ (stored 0%)
